In [1]:
import numpy as np
from utils import *
from utils import *
from dgp import sample_network_chain, get_graph, sample_Y1, agcEffect
import numpy as np


def prepare(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree):
    # 1. Simulate or load data
    adj1 = get_graph(sample_size, min_degree, max_degree, seed=1)
    print(adj1.sum(axis=1).min(), adj1.sum(axis=1).max())

    tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
    rho = np.array([[0,0.1,0.2],
                    [0.1,0,0.1],
                    [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
    nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
    gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1])    # shape (8,)   
    beta = np.array([-1*min_degree,2,-0.2,2,0.1,-2,0.1,2,0.1,0.1])  # shape (10,)

    res_truth1 = agcEffect(
        adj1,
        tau, rho, nu, beta,
        treatment_allocation=0.7,
        R=100,
        burnin_R=100,
        seed=0
    )
    res_truth2 = agcEffect(
        adj1,
        tau, rho, nu, beta,
        treatment_allocation=0.3,
        R=100,
        burnin_R=100,
        seed=0
    )

    ground_truth1 = {}
    ground_truth1['average'] = res_truth1['average']
    ground_truth1['direct'] = res_truth1['direct_effect']
    ground_truth1['indirect'] = res_truth1['psi_0_gamma'] - res_truth2['psi_0_gamma']
    ground_truth1['spillover_effect'] = res_truth1['spillover_effect']
    ground_truth1['psi_1_gamma'] = res_truth1['psi_1_gamma']
    ground_truth1['psi_0_gamma'] = res_truth1['psi_0_gamma']
    ground_truth1['psi_zero'] = res_truth1['psi_zero']

    return adj1, None, None, None, ground_truth1

num_sample = 300
num_burn = 20
n_cpu = 10

In [5]:
sample_size = 800
min_degree = 1
max_degree = 2

adj1, Y_chain, A_chain, L_chain, ground_truth1 = prepare(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)

1 2


100%|██████████| 200/200 [00:00<00:00, 282.86it/s]


In [6]:
results1 = np.load(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_None.npy', allow_pickle=True)
results2 = np.load(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_None.npy', allow_pickle=True)

compute_stats(results1, results2, ground_truth1, adj1)

100%|██████████| 100/100 [00:08<00:00, 12.47it/s]


{'columns': ['average',
  'direct',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'coverage_rate': array([0.89, 0.96, 0.96, 0.91, 0.94, 0.92, 0.94]),
 'bias': array([ 0.00224997, -0.01172232, -0.00079086, -0.00253179, -0.00402703,
         0.0076953 ,  0.01022708]),
 'mse': array([0.00281682, 0.00565636, 0.00524515, 0.00664865, 0.00440579,
        0.00342793, 0.00497325]),
 'var': array([0.00281175, 0.00551895, 0.00524453, 0.00664224, 0.00438957,
        0.00336871, 0.00486865]),
 'ci_length': array([0.17202711, 0.30631985, 0.23066412, 0.29792655, 0.23170333,
        0.19471945, 0.24349532]),
 'true_effect': array([ 0.59675  ,  0.3241125, -0.0245   , -0.0437125,  0.6959375,
         0.371825 ,  0.4155375])}

In [7]:
results1 = np.load(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_outcome.npy', allow_pickle=True)
results2 = np.load(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_outcome.npy', allow_pickle=True)

compute_stats(results1, results2, ground_truth1, adj1)

100%|██████████| 100/100 [00:08<00:00, 12.03it/s]


{'columns': ['average',
  'direct',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'coverage_rate': array([0.83, 0.89, 0.95, 0.98, 0.87, 0.87, 0.91]),
 'bias': array([ 0.01872764,  0.03167437, -0.0033514 , -0.00334843,  0.02160547,
        -0.0100689 , -0.00672047]),
 'mse': array([0.00418968, 0.01164218, 0.00797443, 0.01184849, 0.00625755,
        0.00588154, 0.00825625]),
 'var': array([0.00383896, 0.01063891, 0.00796319, 0.01183728, 0.00579075,
        0.00578016, 0.00821108]),
 'ci_length': array([0.23188346, 0.45249005, 0.34391453, 0.43647874, 0.32175071,
        0.2826701 , 0.34854968]),
 'true_effect': array([ 0.59675  ,  0.3241125, -0.0245   , -0.0437125,  0.6959375,
         0.371825 ,  0.4155375])}

'mse': array([0.00648455, 0.09287534, 0.00135583, 0.00310269, 0.02833928,
        0.01980392, 0.01996758]),

In [8]:
results1 = np.load(f'run/sim_results/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_None.npy')
results2 = np.load(f'run/sim_results/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_None.npy')

compute_stats_ag(results1, results2, ground_truth1)

{'columns': ['average',
  'direct_effect',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'bias': array([ 0.00762525, -0.0012645 , -0.0014645 , -0.0055815 ,  0.00485525,
         0.00611975,  0.01170125]),
 'mse': array([0.00145135, 0.00273038, 0.0030888 , 0.00264082, 0.0019647 ,
        0.00208478, 0.00151653]),
 'var': array([0.0013932 , 0.00272878, 0.00308665, 0.00260966, 0.00194112,
        0.00204733, 0.00137961]),
 'ground_truth': array([ 0.59675  ,  0.3241125, -0.0245   , -0.0437125])}

In [9]:
results1 = np.load(f'run/sim_results/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_outcome.npy')
results2 = np.load(f'run/sim_results/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_outcome.npy')

compute_stats_ag(results1, results2, ground_truth1)

{'columns': ['average',
  'direct_effect',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'bias': array([ 0.078251  ,  0.30050075,  0.0117305 ,  0.00483825,  0.16678325,
        -0.1337175 , -0.13855575]),
 'mse': array([0.00648455, 0.09287534, 0.00135583, 0.00310269, 0.02833928,
        0.01980392, 0.01996758]),
 'var': array([0.00036133, 0.00257464, 0.00121823, 0.00307928, 0.00052263,
        0.00192355, 0.00076988]),
 'ground_truth': array([ 0.59675  ,  0.3241125, -0.0245   , -0.0437125])}

In [2]:
sample_size = 800
min_degree = 2
max_degree = 5

adj1, Y_chain, A_chain, L_chain, ground_truth1 = prepare(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)

2 5


100%|██████████| 200/200 [00:00<00:00, 288.22it/s]


In [6]:
results1 = np.load(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_None.npy', allow_pickle=True)
results2 = np.load(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_None.npy', allow_pickle=True)

compute_stats(results1, results2, ground_truth1, adj1)

100%|██████████| 100/100 [00:15<00:00,  6.61it/s]


{'columns': ['average',
  'direct',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'coverage_rate': array([0.91, 0.93, 0.91, 0.89, 0.9 , 0.91, 0.89]),
 'bias': array([ 2.84185714e-03, -8.72110168e-04, -6.59629949e-03, -2.51176574e-02,
        -9.57340277e-04, -8.52301089e-05,  2.50324273e-02]),
 'mse': array([0.00245302, 0.00693182, 0.00585081, 0.0419332 , 0.00423907,
        0.00349083, 0.03668057]),
 'var': array([0.00244495, 0.00693106, 0.00580729, 0.04130231, 0.00423815,
        0.00349082, 0.03605394]),
 'ci_length': array([0.15970728, 0.29400767, 0.26158896, 0.57688821, 0.20817498,
        0.19668013, 0.53473481]),
 'true_effect': array([ 0.4657375,  0.3202875, -0.0436625, -0.079675 ,  0.564975 ,
         0.2446875,  0.3243625])}

In [7]:
results1 = np.load(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_outcome.npy', allow_pickle=True)
results2 = np.load(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_outcome.npy', allow_pickle=True)

compute_stats(results1, results2, ground_truth1, adj1)

100%|██████████| 100/100 [00:10<00:00,  9.25it/s]


{'columns': ['average',
  'direct',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'coverage_rate': array([0.9 , 0.77, 0.93, 0.79, 0.84, 0.71, 0.73]),
 'bias': array([ 0.01893222,  0.08306912,  0.00227797,  0.0035775 ,  0.03910037,
        -0.04396875, -0.04754626]),
 'mse': array([0.00321098, 0.01667188, 0.0077954 , 0.04334676, 0.00696583,
        0.00564903, 0.04203082]),
 'var': array([0.00285255, 0.0097714 , 0.00779021, 0.04333396, 0.00543699,
        0.00371578, 0.03977017]),
 'ci_length': array([0.21467255, 0.38510932, 0.31927802, 0.6781594 , 0.29573282,
        0.2185794 , 0.63737093]),
 'true_effect': array([ 0.4657375,  0.3202875, -0.0436625, -0.079675 ,  0.564975 ,
         0.2446875,  0.3243625])}

'mse': array([0.00927132, 0.09945723, 0.002831  , 0.00377077, 0.03620025,
        0.01625759, 0.0287898 ]),

In [3]:
results1 = np.load(f'run/sim_results/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_None.npy')
results2 = np.load(f'run/sim_results/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_None.npy')

compute_stats_ag(results1, results2, ground_truth1)

{'columns': ['average',
  'direct_effect',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'bias': array([ 0.00703925, -0.00516525, -0.0077545 ,  0.0028995 ,  0.0022085 ,
         0.00737375,  0.00447425]),
 'mse': array([0.00159099, 0.00209447, 0.00257326, 0.00368117, 0.00182494,
        0.00219345, 0.00280906]),
 'var': array([0.00154144, 0.00206779, 0.00251313, 0.00367276, 0.00182007,
        0.00213908, 0.00278904]),
 'ground_truth': array([ 0.4657375,  0.3202875, -0.0436625, -0.079675 ])}

In [4]:
results1 = np.load(f'run/sim_results/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_outcome.npy')
results2 = np.load(f'run/sim_results/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_outcome.npy')

compute_stats_ag(results1, results2, ground_truth1)

{'columns': ['average',
  'direct_effect',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'bias': array([ 0.093676  ,  0.31062575,  0.01271125,  0.04433725,  0.1873835 ,
        -0.12324225, -0.1675795 ]),
 'mse': array([0.00927132, 0.09945723, 0.002831  , 0.00377077, 0.03620025,
        0.01625759, 0.0287898 ]),
 'var': array([0.00049613, 0.00296887, 0.00266942, 0.00180498, 0.00108767,
        0.00106894, 0.00070691]),
 'ground_truth': array([ 0.4657375,  0.3202875, -0.0436625, -0.079675 ])}

In [10]:
sample_size = 800
min_degree = 3
max_degree = 10

adj1, Y_chain, A_chain, L_chain, ground_truth1 = prepare(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)

3 10


100%|██████████| 200/200 [00:00<00:00, 344.01it/s]


In [11]:
results1 = np.load(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_None.npy', allow_pickle=True)
results2 = np.load(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_None.npy', allow_pickle=True)

compute_stats(results1, results2, ground_truth1, adj1)

100%|██████████| 100/100 [00:26<00:00,  3.74it/s]


{'columns': ['average',
  'direct',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'coverage_rate': array([0.98, 0.94, 0.92, 0.81, 0.96, 0.92, 0.72]),
 'bias': array([ 0.00601181, -0.01561433,  0.00655396,  0.02289216, -0.00351835,
         0.01209598, -0.01079618]),
 'mse': array([0.00115104, 0.00382175, 0.0047506 , 0.03523483, 0.00192961,
        0.00186601, 0.03274636]),
 'var': array([0.0011149 , 0.00357794, 0.00470764, 0.03471078, 0.00191723,
        0.0017197 , 0.03262981]),
 'ci_length': array([0.13314998, 0.22831509, 0.23719074, 0.52303683, 0.17391574,
        0.1482426 , 0.48665736]),
 'true_effect': array([ 0.34095  ,  0.27945  , -0.0546625, -0.10395  ,  0.4295875,
         0.1501375,  0.2540875])}

In [12]:
results1 = np.load(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_outcome.npy', allow_pickle=True)
results2 = np.load(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_outcome.npy', allow_pickle=True)

compute_stats(results1, results2, ground_truth1, adj1)

100%|██████████| 100/100 [00:28<00:00,  3.52it/s]


{'columns': ['average',
  'direct',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'coverage_rate': array([0.98, 0.84, 0.72, 0.38, 0.95, 0.65, 0.27]),
 'bias': array([ 0.00425693,  0.05936097,  0.04688527,  0.08878741,  0.01899774,
        -0.04036322, -0.12915063]),
 'mse': array([0.00143764, 0.00845642, 0.00717141, 0.05123733, 0.00296438,
        0.0035515 , 0.06065007]),
 'var': array([0.00141952, 0.00493269, 0.00497319, 0.04335412, 0.00260346,
        0.00192231, 0.04397019]),
 'ci_length': array([0.1732421 , 0.29928549, 0.25442299, 0.54917457, 0.23406601,
        0.17208981, 0.49290969]),
 'true_effect': array([ 0.34095  ,  0.27945  , -0.0546625, -0.10395  ,  0.4295875,
         0.1501375,  0.2540875])}

 'mse': array([0.00648124, 0.06790836, 0.00632861, 0.00848844, 0.027409  ,
        0.00935543, 0.03464694]),

In [13]:
results1 = np.load(f'run/sim_results/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_None.npy')
results2 = np.load(f'run/sim_results/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_None.npy')

compute_stats_ag(results1, results2, ground_truth1)

{'columns': ['average',
  'direct_effect',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'bias': array([ 0.00940625, -0.013274  , -0.016999  ,  0.0151535 ,  0.0020525 ,
         0.0153265 ,  0.000173  ]),
 'mse': array([0.00121276, 0.00287916, 0.00356269, 0.0040208 , 0.00166142,
        0.00192761, 0.00369998]),
 'var': array([0.00112429, 0.00270296, 0.00327372, 0.00379117, 0.00165721,
        0.00169271, 0.00369995]),
 'ground_truth': array([ 0.34095  ,  0.27945  , -0.0546625, -0.10395  ])}

In [14]:
results1 = np.load(f'run/sim_results/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_outcome.npy')
results2 = np.load(f'run/sim_results/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_outcome.npy')

compute_stats_ag(results1, results2, ground_truth1)

{'columns': ['average',
  'direct_effect',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'bias': array([ 0.0785095 ,  0.254531  ,  0.01650775,  0.090351  ,  0.159463  ,
        -0.095068  , -0.185419  ]),
 'mse': array([0.00648124, 0.06790836, 0.00632861, 0.00848844, 0.027409  ,
        0.00935543, 0.03464694]),
 'var': array([0.0003175 , 0.00312233, 0.00605611, 0.00032514, 0.00198055,
        0.0003175 , 0.00026673]),
 'ground_truth': array([ 0.34095  ,  0.27945  , -0.0546625, -0.10395  ])}